In [ ]:
# Standard library
import os

# Third-party
import astropy.time as atime
from astropy import log as logger
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import gridspec
import numpy as np
import corner

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.units import default_units
from thejoker.util import quantity_from_hdf5
from thejoker.celestialmechanics import OrbitalParams, SimulatedRVOrbit, rv_from_elements

from figurehelpers import samples_units, make_rv_curve_figure, apw_corner

plt.style.use('../thejoker/thejoker.mplstyle')

In [ ]:
name = "HIP22263"

In [ ]:
tbl = np.genfromtxt("../data/HIP22263_result.vels")

In [ ]:
data = RVData(t=atime.Time(tbl[:,0], format='jd', scale='utc'), 
              rv=tbl[:,1]*u.m/u.s, stddev=tbl[:,2]*u.m/u.s)

In [ ]:
data.to_hdf5("../data/HIP22263.hdf5")

In [ ]:
data.plot()

In [ ]:
_path = os.path.join(paths.cache, 'HIP22263.h5')
with h5py.File(_path, 'r') as g:
    pars = OrbitalParams.from_hdf5(g)

In [ ]:
len(pars)

In [ ]:
t_grid = np.linspace(data.t.mjd.min(), data.t.mjd.max(), 1024)

fig,ax = plt.subplots(1,1,figsize=(12,5))

ax = data.plot(ax=ax)

for i in range(len(pars))[:128]:
    orbit = pars.rv_orbit(index=i)
    rv = orbit.generate_rv_curve(t_grid)
    ax.plot(t_grid, rv.to(u.m/u.s), marker='', alpha=0.2)

In [ ]:
plt.hist(pars.P, bins=np.logspace(0,2.,32))
plt.xscale('log')

In [ ]:
plt.hist(pars.ecc)

In [ ]:
plt.hist(pars.jitter)

In [ ]:
_samples_units = samples_units.copy()
_samples_units['v0'] = u.m/u.s
_samples_units['K'] = u.m/u.s

In [ ]:
samples = pars.pack(plot_transform=True, units=_samples_units)

In [ ]:
# Make a corner plot of all samples
_med_v0 = np.median(samples[:,-1])
_mad_v0 = np.median(np.abs(samples[:,-1] - _med_v0))
ranges = [(0,8), (-0.025, 1.025), (0,360), (0,360), (1,3),
          (3.5, 50), (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)]
labels = OrbitalParams.get_labels(_samples_units)

corner_style = dict(alpha=0.25, markersize=1., bins=16, color='#555555')

In [ ]:
fig = apw_corner(samples, range=ranges, labels=labels, **corner_style)
fig.subplots_adjust(left=0.08, bottom=0.08)
fig.suptitle(name, fontsize=24)
fig.savefig(os.path.join(paths.plots, '{}-corner.pdf'.format(name)), dpi=128)

In [ ]:
med_P = np.median(pars.P)

In [ ]:
t_grid = np.linspace(data.t.mjd.min(), data.t.mjd.max(), 1024)

fig,ax = plt.subplots(1,1,figsize=(8,6))

ax.errorbar((data.t.tcb.mjd*u.day/med_P)%1., data.rv.to(u.m/u.s).value, data.stddev.to(u.m/u.s).value,
            linestyle='none', marker='.', color='k')

ax.errorbar((data.t.tcb.mjd*u.day/med_P)%1. + 1., data.rv.to(u.m/u.s).value, data.stddev.to(u.m/u.s).value,
            linestyle='none', marker='.', color='k')

ax.set_xlim(-0.1, 2.1)

ax.set_xlabel(r"$\phi/(2\,\pi)$")
ax.set_ylabel(r"RV [${\rm m}\,{\rm s}^{-1}$]")

ax.set_title("$P = {:.2f}$ day".format(med_P.to(u.day).value))

fig.savefig(os.path.join(paths.plots, '{}-phased.pdf'.format(name)), dpi=128)